### Installation

After the installation is done, make sure to restart the session on Google Colab

In [ ]:
!pip install openagi

In [ ]:
from openagi.agent import Admin
from openagi.llms import OpenAIModel
from openagi.memory import Memory
from openagi.planner.task_decomposer import TaskPlanner
from openagi.worker import Worker

### Get your OpenAI API key

In [ ]:
import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI key:")

Enter your OpenAI key:··········


### Configure the Large Language Model

Documentation of another supported LLMs: [openagi.aiplanet.com/llms](https://openagi.aiplanet.com/components/llm)

Note: Its recommended to have OpenAI or Azure OpenAI for better quality

In [ ]:
config = OpenAIModel.load_from_env_config()
llm = OpenAIModel(config=config)

### Multi-Agent Architecture

**Define Workers**

Workers are special type of classes, responsible for carrying out the tasks assigned by the class "Admin". They utilize tools such as internet search engines, LLMs, and document writers to perform their tasks. Additionally, they can determine which tools to use from a predefined set.

In [ ]:
from openagi.actions.files import WriteFileAction, ReadFileAction
from openagi.actions.tools.ddg_search import DuckDuckGoSearch
from openagi.actions.tools.webloader import WebBaseContextTool

Similarly to how a large task like writing a blog is decomposed into smaller steps such as researching, drafting, and publishing, the admin can define a large task and split it into smaller tasks that are then assigned to the workers.


The primary components, ``TaskWorker``, provide a structured way to define and execute tasks. ``The TaskWorker`` class specializes in executing specific tasks assigned by the planner.

In [ ]:
feedback_collector = Worker(
        role="Customer Feedback Collector",
        instructions="Gather customer feedback specifically about AirPods Pro from various online platforms, including social media, review sites, and forums. Focus on identifying common themes and sentiments related to this product.",
        actions=[DuckDuckGoSearch, WebBaseContextTool, WriteFileAction],
    )

data_analyst = Worker(
    role="Data Analyst",
    instructions="Analyze the collected customer feedback data related to AirPods Pro to identify key trends, recurring issues, and overall customer sentiment. Use statistical tools to quantify the data and provide actionable insights.",
    actions=[ReadFileAction, DuckDuckGoSearch, WebBaseContextTool, WriteFileAction],
)

report_creator = Worker(
    role="Report Creator",
    instructions="Develop a comprehensive customer feedback analysis report based on the data analysis for AirPods Pro. Highlight key findings, trends, and recommendations for improving the product. Ensure the report is well-structured and visually appealing.",
    actions=[ReadFileAction, DuckDuckGoSearch, WebBaseContextTool, WriteFileAction],
)

### Define Admin Agent

Imagine Admin as the master task executor who is responsible for all the major configurations for the execution. From planning of tasks to execution, and defining the brain which is what LLM to use and whether or not to use memory.

Admin is the decision-maker that understand the specifications of tasks and execute them in a more human-like manner.

In [ ]:
admin = Admin(
        actions=[DuckDuckGoSearch],
        planner=TaskPlanner(human_intervene=False),
        memory=Memory(),
        llm=llm,
    )
admin.assign_workers([feedback_collector, data_analyst, report_creator])

## Run the Admin

In [ ]:
res = admin.run(
        query="Create a customer feedback analysis report for AirPods Pro.",
        description="Collect and analyze customer feedback specifically for AirPods Pro from multiple online sources. Identify common themes, recurring issues, and overall customer sentiment. Develop a comprehensive report that provides actionable insights and recommendations for improving the AirPods Pro. Ensure the report is detailed, well-organized, and visually appealing.",
    )

In [ ]:
print(res)